# Feedback Prize - Evaluating Student Writing

このコンペの説明は少し分かりづらいように思えます。
冬休みに取り組みたい日本人向けに、自習もかねて解説してみました。

参考になった方は、ぜひ投票をお願いします。
<br><br>
I referred to the following great notebook.<br>
<a href="https://www.kaggle.com/erikbruin/nlp-on-student-writing-eda">NLP on Student Writing: EDA</a><br>
<a href="https://www.kaggle.com/odins0n/feedback-prize-eda">🔥📊 Feedback Prize - EDA 📊🔥</a>

In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
from matplotlib.ticker import FuncFormatter
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
import spacy

In [ ]:
train = pd.read_csv('../input/feedback-prize-2021/train.csv')
train[['discourse_id', 'discourse_start', 'discourse_end']] = train[['discourse_id', 'discourse_start', 'discourse_end']].astype(int)

sample_submission = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

#The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell
train_txt = glob('../input/feedback-prize-2021/train/*.txt') 
test_txt = glob('../input/feedback-prize-2021/test/*.txt')

# コンペの紹介

データセットには、6年生から12年生の米国の学生が書いた論争のエッセイが含まれています。エッセイは、論争的な執筆で一般的に見られる要素について専門家の評価者によって注釈が付けられました。

これはコードの競争であり、目に見えないテストセットに対して実行されるコードを提出することに注意してください。見えないテストセットは約1万ドキュメントです。ノートブックをテストするための小さな公開テストサンプルが提供されています。

あなたの仕事は人間の注釈を予測することです。まず、各エッセイを個別の修辞的要素と議論的要素（つまり、談話要素）に分割してから、各要素を次のいずれかに分類する必要があります。

- Lead リード-統計、引用、説明、または読者の注意を引き、論文に向けるその他のデバイスで始まる紹介
- Position -立場-主な質問に対する意見または結論
- Claim -主張-立場を支持する主張
- Counterclaim -反訴-別の主張に反論する、またはその立場に反対の理由を与える主張
- Rebuttal -反訴-反訴に反論する主張
- Evidence -証拠-主張、反訴、または反論を裏付けるアイデアまたは例。
- Concluding Statement -結論ステートメント-クレームを言い換える結論ステートメント

まず、1つのエッセイの全文を見てみましょう。

In [ ]:
!cat ../input/feedback-prize-2021/train/423A1CA112E2.txt

trainデータは、このエッセイから抽出された次の人間の注釈を提供します。

In [ ]:
train.query('id == "423A1CA112E2"')

トレーニングセットは、.txtファイルのフォルダー内の個々のエッセイと、これらのエッセイの注釈付きバージョンを含む.csvファイルで構成されます。 エッセイの一部には注釈が付けられていないことに注意することが重要です（つまり、上記の分類の1つに当てはまりません）。

train.csv-トレーニングセット内のすべてのエッセイの注釈付きバージョンを含む.csvファイル
- id-エッセイ応答のIDコード
- discourse_id-談話要素のIDコード
- discourse_start-エッセイの応答で談話要素が始まる文字の位置
- discourse_end-談話要素がエッセイ応答で終了する文字位置
- discourse_text-談話要素のテキスト
- discourse_type-談話要素の分類
- discourse_type_num-談話要素の列挙されたクラスラベル
- predictionstring-予測に必要なトレーニングサンプルの単語インデックス

ここでの正解は、談話タイプと予測文字列の組み合わせです。予測文字列はエッセイの単語のインデックスに対応し、この一連の単語の予測される談話タイプは正しいはずです。正しい談話タイプが予測されているが、Ground Truthで指定されているよりも長いまたは短い単語のシーケンスである場合、部分的に一致する可能性があります。

ご覧のとおり、エッセイのすべてのテキストが談話の一部であるとは限りません。この場合、タイトルは談話の一部ではありません。

# discourse_typeごとの長さと頻度および相対位置

談話の長さとクラス（discourse_type）の間に相関関係はありますか？はいあります。証拠は、平均して最長の割引タイプです。発生頻度を見ると、反訴と反論は比較的まれであることがわかります

In [ ]:
#add columns
train["discourse_len"] = train["discourse_text"].apply(lambda x: len(x.split()))
train["pred_len"] = train["predictionstring"].apply(lambda x: len(x.split()))


cols_to_display = ['discourse_id', 'discourse_text', 'discourse_type','predictionstring', 'discourse_len', 'pred_len']
train[cols_to_display].head()

In [ ]:
fig = plt.figure(figsize=(12,8))

ax1 = fig.add_subplot(211)
ax1 = train.groupby('discourse_type')['discourse_len'].mean().sort_values().plot(kind="barh")
ax1.set_title("Average number of words versus Discourse Type", fontsize=14, fontweight = 'bold')
ax1.set_xlabel("Average number of words", fontsize = 10)
ax1.set_ylabel("")

ax2 = fig.add_subplot(212)
ax2 = train.groupby('discourse_type')['discourse_type'].count().sort_values().plot(kind="barh")
ax2.get_xaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ','))) #add thousands separator
ax2.set_title("Frequency of Discourse Type in all essays", fontsize=14, fontweight = 'bold')
ax2.set_xlabel("Frequency", fontsize = 10)
ax2.set_ylabel("")

plt.tight_layout(pad=2)
plt.show()

フィールドdiscourse_type_numがあります。 Evidence1、Position1、Claim1はほとんどの場合エッセイに含まれていることがわかります。 ほとんどの学生はまた、少なくとも1つの結論ステートメントを持っていました。 

In [ ]:
fig = plt.figure(figsize=(12,8))
av_per_essay = train['discourse_type_num'].value_counts(ascending = True).rename_axis('discourse_type_num').reset_index(name='count')
av_per_essay['perc'] = round((av_per_essay['count'] / train.id.nunique()),3)
av_per_essay = av_per_essay.set_index('discourse_type_num')
ax = av_per_essay.query('perc > 0.03')['perc'].plot(kind="barh")
ax.set_title("discourse_type_num: Percent present in essays", fontsize=20, fontweight = 'bold')
ax.bar_label(ax.containers[0], label_type="edge")
ax.set_xlabel("Percent")
ax.set_ylabel("")
plt.show()

However, I am also interested in the relative positions of discourse types with the essays. Therefore, I am adding this number in the loop below. I think the main takeaway is that the Lead (if it's there!) is almost always the first discourse in an essay.

**More on this in the next version.**

In [ ]:
train['discourse_nr'] = 1
counter = 1

for i in tqdm(range(1, len(train))):
    if train.loc[i, 'id'] == train.loc[i-1, 'id']:
        counter += 1
        train.loc[i, 'discourse_nr'] = counter
    else:
        counter = 1
        train.loc[i, 'discourse_nr'] = counter
        
av_position = train.groupby('discourse_type')['discourse_nr'].mean().sort_values()
av_position

# 注釈（discourse_textとして使用されていないテキスト）間のギャップを調査する

trainの中で最後のdiscourse_endを取るだけでは、最後のテキストが談話として使用されていない可能性があるため、完全に正しいわけではありません。

In [ ]:
# this code chunk is copied from Rob Mulla
len_dict = {}
word_dict = {}
for t in tqdm(train_txt):
    with open(t, "r") as txt_file:
        myid = t.split("/")[-1].replace(".txt", "")
        data = txt_file.read()
        mylen = len(data.strip())
        myword = len(data.split())
        len_dict[myid] = mylen
        word_dict[myid] = myword
train["essay_len"] = train["id"].map(len_dict)
train["essay_words"] = train["id"].map(word_dict)

In [ ]:
#initialize column
train['gap_length'] = np.nan

#set the first one
train.loc[0, 'gap_length'] = 7 #discourse start - 1 (previous end is always -1)

#loop over rest
for i in tqdm(range(1, len(train))):
    #gap if difference is not 1 within an essay
    if ((train.loc[i, "id"] == train.loc[i-1, "id"])\
        and (train.loc[i, "discourse_start"] - train.loc[i-1, "discourse_end"] > 1)):
        train.loc[i, 'gap_length'] = train.loc[i, "discourse_start"] - train.loc[i-1, "discourse_end"] - 2
        #minus 2 as the previous end is always -1 and the previous start always +1
    #gap if the first discourse of an new essay does not start at 0
    elif ((train.loc[i, "id"] != train.loc[i-1, "id"])\
        and (train.loc[i, "discourse_start"] != 0)):
        train.loc[i, 'gap_length'] = train.loc[i, "discourse_start"] -1


 #is there any text after the last discourse of an essay?
last_ones = train.drop_duplicates(subset="id", keep='last')
last_ones['gap_end_length'] = np.where((last_ones.discourse_end < last_ones.essay_len),\
                                       (last_ones.essay_len - last_ones.discourse_end),\
                                       np.nan)

cols_to_merge = ['id', 'discourse_id', 'gap_end_length']
train = train.merge(last_ones[cols_to_merge], on = ["id", "discourse_id"], how = "left")

In [ ]:
#display an example
cols_to_display = ['id', 'discourse_start', 'discourse_end', 'discourse_type', 'essay_len', 'gap_length', 'gap_end_length']
train[cols_to_display].query('id == "AFEC37C2D43F"')

In [ ]:
#how many pieces of tekst are not used as discourses?
print(f"Besides the {len(train)} discourse texts, there are {len(train.query('gap_length.notna()', engine='python'))+ len(train.query('gap_end_length.notna()', engine='python'))} pieces of text not classified.")

以下に、外れ値を取り除いたすべてのギャップの長さのヒストグラムを示します（すべてのギャップが300文字より長い）。

In [ ]:
all_gaps = (train.gap_length[~train.gap_length.isna()]).append((train.gap_end_length[~train.gap_end_length.isna()]), ignore_index= True)
#filter outliers
all_gaps = all_gaps[all_gaps<300]
fig = plt.figure(figsize=(12,6))
all_gaps.plot.hist(bins=100)
plt.title("Histogram of gap length (gaps up to 300 characters only)")
plt.xticks(rotation=0)
plt.xlabel("Length of gaps in characters")
plt.show()

# エッセイの分類ごと色塗り

エッセイに含まれる談話を分類ごとに色塗りします。無分類の箇所もあることに注意してください。

In [ ]:
def add_gap_rows(essay):
    cols_to_keep = ['discourse_start', 'discourse_end', 'discourse_type', 'gap_length', 'gap_end_length']
    df_essay = train.query('id == @essay')[cols_to_keep].reset_index(drop = True)

    #index new row
    insert_row = len(df_essay)
   
    for i in range(1, len(df_essay)):          
        if df_essay.loc[i,"gap_length"] >0:
            if i == 0:
                start = 0 #as there is no i-1 for first row
                end = df_essay.loc[0, 'discourse_start'] -1
                disc_type = "Nothing"
                gap_end = np.nan
                gap = np.nan
                df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end]
                insert_row += 1
            else:
                start = df_essay.loc[i-1, "discourse_end"] + 1
                end = df_essay.loc[i, 'discourse_start'] -1
                disc_type = "Nothing"
                gap_end = np.nan
                gap = np.nan
                df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end]
                insert_row += 1

    df_essay = df_essay.sort_values(by = "discourse_start").reset_index(drop=True)

    #add gap at end
    if df_essay.loc[(len(df_essay)-1),'gap_end_length'] > 0:
        start = df_essay.loc[(len(df_essay)-1), "discourse_end"] + 1
        end = start + df_essay.loc[(len(df_essay)-1), 'gap_end_length']
        disc_type = "Nothing"
        gap_end = np.nan
        gap = np.nan
        df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end]
        
    return(df_essay)

In [ ]:
add_gap_rows("129497C3E0FC")

In [ ]:
def print_colored_essay(essay):
    df_essay = add_gap_rows(essay)
    #code from https://www.kaggle.com/odins0n/feedback-prize-eda, but adjusted to df_essay
    essay_file = "../input/feedback-prize-2021/train/" + essay + ".txt"

    ents = []
    for i, row in df_essay.iterrows():
        ents.append({
                        'start': int(row['discourse_start']), 
                         'end': int(row['discourse_end']), 
                         'label': row['discourse_type']
                    })

    with open(essay_file, 'r') as file: data = file.read()

    doc2 = {
        "text": data,
        "ents": ents,
    }

    colors = {'Lead': '#EE11D0','Position': '#AB4DE1','Claim': '#1EDE71','Evidence': '#33FAFA','Counterclaim': '#4253C1','Concluding Statement': 'yellow','Rebuttal': 'red'}
    options = {"ents": df_essay.discourse_type.unique().tolist(), "colors": colors}
    spacy.displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True);

In [ ]:
print_colored_essay("7330313ED3F0")
#print_colored_essay("423A1CA112E2")

# 分類ごとに最もよく使用される単語¶
ここで、各discourse_type（ストップワードを除く）で最もよく使用されている単語を調べたいと思います。 また、各discourse_typeの図のいたるところにある余分な単語をいくつか取り出しました。

In [ ]:
train['discourse_text'] = train['discourse_text'].str.lower()

#get stopwords from nltk library
stop_english = stopwords.words("english")
other_words_to_take_out = ['school', 'students', 'people', 'would', 'could', 'many']
stop_english.extend(other_words_to_take_out)

#put series of Top-10 words in dict for all discourse types
counts_dict = {}
for dt in train['discourse_type'].unique():
    df = train.query('discourse_type == @dt')
    text = df.discourse_text.apply(lambda x: x.split()).tolist()
    text = [item for elem in text for item in elem]
    df1 = pd.Series(text).value_counts().to_frame().reset_index()
    df1.columns = ['Word', 'Frequency']
    df1 = df1[~df1.Word.isin(stop_english)].head(10)
    df1 = df1.set_index("Word").sort_values(by = "Frequency", ascending = True) #to series
    counts_dict[dt] = df1


In [ ]:
plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.5)

keys = list(counts_dict.keys())

for n, key in enumerate(keys):
    ax = plt.subplot(4, 2, n + 1)
    ax.set_title(f"Most used words in {key}")
    counts_dict[keys[n]].plot(ax=ax, kind = 'barh')
    plt.ylabel("")